Notre objectif dans ce Notebook est d'entrainer un GCN pour prédire les genres musicaux de prédilection d'un utilisateur en fonction de ses connexions dans le graphe. Nous utiliserons enusite les embeddings produit par le modèle pour déterminer des clusters et juger de leur qualité par rapport à d'autres approches de détection de communautés. 

In [82]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils
from torch.utils.data import Dataset

import time
from datetime import datetime

import networkx as nx
import numpy as np
import pandas as pd 
import json
import torch
import torch.optim as optim

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.transforms as T

# from torch.utils.tensorboard import SummaryWriter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

On commence par mettre le dataset des préférences deezer en Croatie dans un format utilisable par un réseau de neurone DGL (bibliothèque python pour les réseaux de neurones sur graphes)

In [2]:
#Preparons les données sur les noeuds pour qu'elles soient utilisables par le modèle

with open('deezer_clean_data/HR_genres.json') as json_file:
    data = json.load(json_file)

# On crée un dictionnaire qui associe à chaque genre un numéro

nodes = list(data.keys())
genres = []

for key, value in data.items():
    for genre in value :
        if genre not in genres:
            genres.append(genre)

# On crée un dictionnaire qui associe à chaque noeud un vecteur de 0 et de 1 en fonction de ses genres

dico = {}

for key, value in data.items():
    vect = [0]*len(genres)
    for genre in value:
        vect[genres.index(genre)] = 1
    dico[key] = vect

# On ordonne les clés en ordre croissant 

dico = dict(sorted(dico.items()))

# On transforme le dictionnaire en array numpy

X = np.array(list(dico.values()))

# On supprime les noeuds qui ne sont reliés avec aucun autre noeud

edges_data = pd.read_csv("deezer_clean_data/HR_edges.csv")

# Assuming your CSV has two columns named 'source' and 'target' representing edges
edges = [(row['node_1'], row['node_2']) for index, row in edges_data.iterrows()]

# Construct the graph from the edge data
G = nx.Graph()
G.add_edges_from(edges)

# Calculate the degree of each node
degrees = dict(G.degree())

# Identify nodes with zero degree
zero_degree_nodes = [node for node, degree in degrees.items() if degree == 0]

print(zero_degree_nodes)

#Remove zero degree nodes from X

for i in zero_degree_nodes:
    X = np.delete(X, i, 0)


X.shape

[]


(54573, 84)

In [3]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import torch
from dgl.data import DGLDataset


class HRDeezerDataset(DGLDataset):
    def __init__(self):
        super().__init__(name="HRDeezer")

    def process(self):
        #nodes_data = pd.read_csv("/members.csv")
        edges_data = pd.read_csv("deezer_clean_data/HR_edges.csv")
        #node_features = torch.from_numpy(np.zeros((X.shape[0], 1)))
        # node_labels = torch.from_numpy(
        #     nodes_data["Club"].astype("category").cat.codes.to_numpy()
        # )
        node_labels = torch.from_numpy(X).float()
        #edge_features = torch.from_numpy(edges_data["Weight"].to_numpy())
        edges_src = torch.from_numpy(edges_data["node_1"].to_numpy())
        edges_dst = torch.from_numpy(edges_data["node_2"].to_numpy())

        self.graph = dgl.graph(
            (edges_src, edges_dst), num_nodes=X.shape[0]
        )
        #self.graph.ndata["feat"] = node_features
        self.graph.ndata["label"] = node_labels
        #self.graph.edata["weight"] = None

        # If your dataset is a node classification dataset, you will need to assign
        # masks indicating whether a node belongs to training, validation, and test set.
        n_nodes = X.shape[0]
        n_train = int(n_nodes * 0.6)
        n_val = int(n_nodes * 0.2)
        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        val_mask = torch.zeros(n_nodes, dtype=torch.bool)
        test_mask = torch.zeros(n_nodes, dtype=torch.bool)
        train_mask[:n_train] = True
        val_mask[n_train : n_train + n_val] = True
        test_mask[n_train + n_val :] = True
        self.graph.ndata["train_mask"] = train_mask
        self.graph.ndata["val_mask"] = val_mask
        self.graph.ndata["test_mask"] = test_mask

    def __getitem__(self, i):
        return self.graph

    def __len__(self):
        return 1
    
    def num_classes(self):
        return X.shape[1]
    
    def num_nodes(self):
        return X.shape[0]


dataset = HRDeezerDataset()
graph = dataset[0]

graph = dgl.add_self_loop(graph)

print(graph)

Graph(num_nodes=54573, num_edges=552775,
      ndata_schemes={'label': Scheme(shape=(84,), dtype=torch.float32), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})


Nous déclarons maintenant notre réseau. Nous commençons par un petit réseau contenant des couches convolutives pour graphes permettant le passage de messages entre les noeuds. 
Les noeuds n'ayant pas de features nous leur associons des embeddings que le réseau va pouvoir apprendre. Les embeddings sont intialement générés aléatoirement.  

In [85]:
from dgl.nn import GraphConv


class GCN(nn.Module):
    def __init__(self, num_nodes, h_feats1, num_classes):
        super(GCN, self).__init__()
        #On créé des embeddings pour les noeuds
        self.node_embedding = nn.Embedding(num_nodes, h_feats1)
        # Initialize the embeddings with small random values
        nn.init.normal_(self.node_embedding.weight, std=4)
        self.conv1 = GraphConv(h_feats1, h_feats1*2, weight=True)
        self.conv2 = GraphConv(h_feats1*2, h_feats1*4, weight=True)
        self.conv3 = GraphConv(h_feats1*4, h_feats1*6, weight=True)
        self.linear = nn.Linear(h_feats1*6, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, g):
        x = self.node_embedding.weight
        x = self.conv1(g, x)
        x = F.relu(x)
        x = self.conv2(g, x)
        x= F.relu(x)
        x = self.conv3(g, x)
        embs = self.linear(x)
        x = self.sigmoid(embs)
        return x, embs

On déclare la fonction d'entrainement du réseau. 

In [86]:
def train_2(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    #features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for e in range(100):
        # Forward
        logits, embs = model(g) 
        
        # Compute prediction
        pred = logits > 0.5
        

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])


        # Compute accuracy on training/validation/test
        # Check for identical rows
        identical_rows = torch.all(torch.eq(pred[train_mask], labels[train_mask]), dim=1)
        num_identical_rows = torch.sum(identical_rows).item()
        train_acc = num_identical_rows / train_mask.sum().item()

        identical_rows = torch.all(torch.eq(pred[val_mask], labels[val_mask]), dim=1)
        num_identical_rows = torch.sum(identical_rows).item()
        val_acc = num_identical_rows / val_mask.sum().item()

        identical_rows = torch.all(torch.eq(pred[test_mask], labels[test_mask]), dim=1)
        num_identical_rows = torch.sum(identical_rows).item()
        test_acc = num_identical_rows / test_mask.sum().item()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print(
                f"In epoch {e}, loss: {loss:.3f}, train acc: {train_acc:.3f} val acc: {val_acc:.3f} (best {best_val_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})"
            )

In [88]:
dataset = HRDeezerDataset()
task = 'node'
model = GCN(dataset.num_nodes(), 32, dataset.num_classes())

print(model)

GCN(
  (node_embedding): Embedding(54573, 32)
  (conv1): GraphConv(in=32, out=64, normalization=both, activation=None)
  (conv2): GraphConv(in=64, out=128, normalization=both, activation=None)
  (conv3): GraphConv(in=128, out=192, normalization=both, activation=None)
  (linear): Linear(in_features=192, out_features=84, bias=True)
  (sigmoid): Sigmoid()
)


In [ ]:
train_2(graph, model)

Problème : la matrice est trop parsemée et l'entrainement pousse le réseau à descendre tous les poids à 0. En effet, les utilisateurs n'ont en général que 2 ou 3 genres musicaux associés sur 84 ce qui fait que les 0 sont largement majoritaires dans les tenseurs associés et que diminuer la loss revient à descendre les poids à 0. C'est ce qu'on observe pendant l'entrainement. Notre réseau est surement trop petit pour saisir la complexité des données. On essaye de faire une prédiction multi-label multi-classe sur 84 classes, ce qui représente une tâche compliquée. 

## Représentation des embeddings sortants du réseau entrainé ##

On va chercher à représenter les embeddings générés par le réseau au cours de l'entrainement. On a pas trop d'espoir de voir une séparation claire étant donnée la qualité des prédictions du réseau. 

In [12]:
#Une fonction pour afficher les représentation en dimensions réduites

def plot_scatter(embeddings, labels, title):
    
    plt.figure(figsize=(10, 6))
    plt.scatter(embeddings[:, 0], embeddings[:, 1], c=labels, cmap='viridis', s=10)
    plt.title(title)
    plt.colorbar()
    plt.show()

#On représente les embeddings en 2D

n_components = 2

In [14]:

embs = []


pred, emb = model(graph)

print(emb.shape)


emb_np = pred.detach().numpy()

print(pred.shape)


tsne = TSNE(n_components=2, random_state=42)
tsne_result = tsne.fit_transform(emb_np)

#We plot the t-SNE representation of the embeddings

plot_scatter(tsne_result, labels=None, title='t-SNE')

torch.Size([54573, 32])
torch.Size([54573, 84])
